In [1]:
import pandas as pd
import numpy as np
cars = pd.read_csv("../data/cars/USA_cars_datasets.csv")
d = dict(cars.brand.value_counts())
remove = [k for k in d if (d[k] < 10)]
for k in remove: del d[k]
cars = cars[cars['brand'].isin(d)]  
cars.head()

def conv(s):
    n = ''
    for i in s:
        if i.isdigit():
            n += i
    if "day" in s:
        return int(n)*24*60
    if "hour" in s:
        return int(n)*60
    if "minute" in s:
        return int(n)
    
cars['condition'] = cars['condition'].apply(conv)
cars = cars[cars.condition.apply(lambda x: isinstance(x,float))]
cars = cars[cars[['price', 'year', 'mileage', 'condition']].notnull().all(1)]
cars.shape

(2448, 13)

In [2]:
cars2 = cars[['price', 'year', 'mileage', 'condition']]
cars2.head()

,price,year,mileage,condition
1,2899,2011,190552.0,8640.0
2,5350,2018,39590.0,2880.0
3,25000,2014,64146.0,1320.0
4,27700,2018,6654.0,1320.0
5,5700,2018,45561.0,2880.0


In [3]:
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split
train, test = train_test_split(cars2, test_size = 0.2)
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, normalize
scale = StandardScaler()


In [20]:
X = train[["year", "mileage", "condition"]]
y = train['price']
X[["year"]] = scale.transform(X[["year"]].values)
print(X)

          year   mileage  condition
777   0.668215   13532.0      360.0
1732  0.668215   11544.0      120.0
1648 -0.215218   54984.0     2880.0
1209 -1.393129   98470.0     2880.0
129   0.079259   59537.0     1260.0
...        ...       ...        ...
2158  0.668215   26821.0     2880.0
2010 -0.509696   62542.0    12960.0
572   0.668215   17684.0     2880.0
2022 -0.509696  126708.0     1260.0
2183  0.373737   41569.0     2880.0

[1958 rows x 3 columns]


In [17]:
est = sm.OLS(y, X).fit()

est.summary()




<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.507
Model:                            OLS   Adj. R-squared (uncentered):              0.506
Method:                 Least Squares   F-statistic:                              668.9
Date:                Fri, 08 May 2020   Prob (F-statistic):                   3.70e-299
Time:                        13:51:36   Log-Likelihood:                         -21694.
No. Observations:                1958   AIC:                                  4.339e+04
Df Residuals:                    1955   BIC:                                  4.341e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
year        9833.9039    411.655     23.889      0.000    9026.576    1.06e+04
mileage        0.1462      0.006     22.802      0.000       0.134       0.159
condition      1.5888      0.082     19.467      0.000       1.429       1.749
==============================================================================
Omnibus:                      420.817   Durbin-Watson:                   1.668
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7724.242
Skew:                           0.505   Prob(JB):                         0.00
Kurtosis:                      12.678   Cond. No.                     8.36e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.36e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
est.predict([scale.fit_transform([[2011]]), 190552, 8640])

array([array([[41578.60146605]])], dtype=object)

In [7]:
def prediction(x,y,z):
    x = scale.fit_transform([[x]])
    scaled = [x,y,z]
    predicted = est.predict(scaled)[0][0][0]
    return(predicted)

prediction_test = cars2.copy()
prediction_test['price2'] = prediction_test.apply(lambda x: prediction(x['year'], x['mileage'], x['condition']), axis=1)
prediction_test['residual'] = abs(prediction_test['price'] - prediction_test['price2'])
prediction_test = prediction_test.reset_index()


In [9]:
prediction_test.residual.mean()


14516.771515167287